In [9]:
import pandas as pd 
import numpy as np
from datetime import datetime

c = ['apple','banana','cat','dog'] #阵列一
d = [123,456,789,56789] #阵列二
sr = pd.Series(d,index = c) # 指定c为index
#提供时间变成index如何？
f=[1,2,3,4,5,6,7,8,9] 
sr_2 = pd.Series(f, pd.date_range(start='2018-05-01', end='2018-05-9')) 
#pd.date_range是内建的日期序列产生器
#如范例，这样你的index就是2018-05-01～2018-05-09共九天
sr

apple       123
banana      456
cat         789
dog       56789
dtype: int64

In [13]:
# OK再来就是厉害的了，可以直接对于Series物件作大小判断，pandas会帮你输出每个栏位的true or false
scores=pd.Series({'小明':90,'小华':80,'小李':70,'小强':55})
print(scores>60)

小明     True
小华     True
小李     True
小强    False
dtype: bool


In [15]:
#比较常用的做法是直接筛选(不符合的资料就不会显示出来) 
scores[scores>60]
# #小强没及格所以不会出现在资料里

小明    90
小华    80
小李    70
dtype: int64

In [17]:
# 而且可以扩充到多个条件
scores[(scores>60)&(scores<90)]

小华    80
小李    70
dtype: int64

In [19]:
#直接计算开根号乘以10也没有问题
new_scores=scores**0.5*10
new_scores

小明    94.868330
小华    89.442719
小李    83.666003
小强    74.161985
dtype: float64

In [21]:
# 一条序列可以处理的资料很有限
# 此时Dataframe就是主角了
# 这边介绍一下怎么建立Dataframe
#最基本的array+dict可直接输入
scores = [{"姓名":"小华","数学":90, "国文":80}, 
          {"姓名":"小明","数学":70 , "国文":55}, 
          {"姓名":"小李","数学":45, "国文":75}] 
score_df = pd.DataFrame(scores)
#纯字典格式->利用from_dict功能
scores = {"姓名":["小华","小明","小李"], 
          "国文":[80,55,75], 
          "数学":[90 ,70,45]} 
score_df = pd.DataFrame.from_dict(scores)
#两个都会产生一样的dataframe 
score_df

,姓名,国文,数学
0,小华,80,90
1,小明,55,70
2,小李,75,45


In [22]:
#其他的方法.
小华= {'数学':90, '国文':80} 
小明= {'数学':70, '国文':55} 
小李= {'数学':45, '国文':75} 
df = pd .DataFrame.from_dict([小华,小明,小李])
df

,数学,国文
0,90,80
1,70,55
2,45,75


In [23]:
# #增加一列的方法
df['姓名']=['小华','小明','小李']
df

,数学,国文,姓名
0,90,80,小华
1,70,55,小明
2,45,75,小李


In [25]:
#如果要把原本的Series转成Dataframe也可以～
scores = pd.Series({'小明':90, '小华':80, '小李':70, '小强':55}) 
#把Series转成Dataframe只需要使用指令就好
score_df=scores.to_frame()
score_df

,0
小明,90
小华,80
小李,70
小强,55


好，再来就要完成这个成绩排序表

我们会设定一个情境，就是假设班级导师

收到了不同科目老师的成绩表，他想要先看一下每科的排序

然后再把分数加总起来，再看总分的排序

所以我们会有三个科目的成绩

假设是来自不同老师的资料，所以会有三条Series

(读取资料的方式最后补充，现在先手动建立）

In [29]:
#我们先读取成绩
math = pd.Series({'小明':90, '小华':80, '小李':70, '小强':55}) 
chinese = pd.Series({'小明' :86, '小华':76, '小李':58, '小强':92}) 
english = pd.Series({'小明':0, '小华':45, '小李':69 , '小强':32})

因为我们要看每个科目的排序，所以要多一列

但是Series的值只有一列，所以我们要用Dataframe

转换之后，再新增一列来储存排序这个数值

In [30]:
#写一个通用的方式，让每条Series都可以用
def have_rank(sr,subject): #传入series与科目名称
    new_sr= sr.to_frame() #转成dataframe 
    new_sr= new_sr.rename(columns={0 : subject}) #给column命名
    new_sr[subject+'_rank'] = new_sr[subject].rank(method='min',ascending=False) 
  #增加一个新列，并且用pd内建的rank这个功能来产生排名
    return new_sr #最后传回这个dataframe

关于rank这个工具
里面的method就是在说同排名是要取最小值、最大值还是平均
然后ascending很明显就是处理升序或降序
所以接下来就是把这几组dataframe都run function
然后合并起来就是完整的成绩表
关于合并，也是一门大学问
先上code
#处理这三个科目

In [31]:
math = have_rank(math, '数学') 
chinese = have_rank(chinese, '国语') 
english = have_rank(english, '英文')
sum_table = pd.concat([math,chinese,english], axis=1) #合并

合并我们这边用concat，基本上如果index处理好的话
我这边就不需要多设定什么参数，concat会帮我自动对应
合并完，再加总分数然后给总排名再排序整个表

In [34]:
sum_table['总分'] = sum_table['数学']+sum_table['国语']+sum_table['英文'] 
#加总
sum_table['总排名'] = sum_table['总分'].rank(method ='min',ascending=False) 
#Rank功能
sum_table = sum_table.sort_values('总排名') 
#sort_values
sum_table

,数学,数学_rank,国语,国语_rank,英文,英文_rank,总分,总排名
小华,80,2.0,76,3.0,45,2.0,201,1.0
小李,70,3.0,58,4.0,69,1.0,197,2.0
小强,55,4.0,92,1.0,32,3.0,179,3.0
小明,90,1.0,86,2.0,0,4.0,176,4.0


In [35]:
sum_table.to_excel('table.xls') #这样就好了